In [0]:
%run ./01-config

In [0]:
%run ./02-setup

In [0]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import lit, current_timestamp, col, regexp_replace, upper, sha2, concat_ws
import requests
import uuid

class Silver():
    def __init__(self):

        print("Loading configuration settings...")
        conf = Config()
        self.nyc_population_by_community_bronze_table = "nyc_population_by_community_raw"
        self.nyc_restaurant_inspection_bronze_table = "nyc_restaurant_inspection_raw"
        self.silver_path = conf.storage_account + "silver_db/"
        self.catalog_name = conf.catalog_name
        self.bronze_db_name = "bronze_db"
        self.db_name = "silver_db"
        print("✅ Configuration Loaded:")
        print(f"   - Storage Path for Silver Layer: {self.silver_path}")
        print(f"   - Catalog Name: {self.catalog_name}")
        print(f"   - Database Name: {self.db_name}")
        print("🚀 Silver Ingestion Initialized Successfully! 🎯\n")

    def table_exists(self, silver_table_name):
        """Check if a Delta table exists in the catalog."""
        try:
            spark.table(silver_table_name)
            return True
        except:
            return False
    
    def dim_borough_population(self):
        """Load NYC Borough Population data into the Silver layer with incremental loading."""
        
        print("🔹 Starting silver layer data ingestion for NYC Borough Population...")

        bronze_table_name = f"{self.catalog_name}.{self.bronze_db_name}.nyc_population_by_community_raw"
        silver_table_name = f"{self.catalog_name}.{self.db_name}.dim_borough_population"
        dim_borough_population_storage_path = self.silver_path + "dim_borough_population/"
        ingestion_timestamp = datetime.utcnow()

        if self.table_exists(silver_table_name):
            print(f"📌 Table {silver_table_name} exists. Performing incremental load...")
            existing_df = spark.table(silver_table_name)
            print(f"🔍 Existing record count: {existing_df.count()}")

            df = spark.read.table(bronze_table_name)
            columns = df.columns
            population_cols = [col_name for col_name in df.columns if "population" in col_name]
            df_unpivoted = df.melt(["borough", "cd_name"], population_cols, "year", "population")
            final_df = df_unpivoted.withColumn("borough", upper(col("borough")))\
                                  .withColumn("cd_name", upper(col("cd_name")))\
                                  .withColumn("year", regexp_replace(col("year"), "^_|_population", ""))\
                                  .withColumn("year", col("year").cast("int"))\
                                  .withColumn("population", col("population").cast("int"))\
                                  .withColumn("source", lit(bronze_table_name))\
                                  .withColumn("ingestion_timestamp", lit(ingestion_timestamp))

            new_df = final_df.join(existing_df,["borough", "cd_name", "year"], "left_anti")
            
            new_records_count = max(new_df.count() - existing_df.count(), 0)
            print(f"🆕 New records to insert: {new_records_count}")

            if new_records_count > 0:
                df.write.format("delta").mode("append").saveAsTable(silver_table_name)
                print(f"✅ {new_records_count} new records inserted into {silver_table_name}.")
            else:
                print("✅ No new records to insert. Data is already up-to-date.")

        else:
            print(f"🛠️ Table {silver_table_name} does not exist. Creating Table...")
            spark.sql(f"""
                           CREATE TABLE IF NOT EXISTS {silver_table_name}(
                               borough STRING,
                               cd_name STRING,
                               year INT,
                               population INT,
                               source STRING,
                               ingestion_timestamp TIMESTAMP)
                            USING DELTA
                            LOCATION '{dim_borough_population_storage_path}'
                        """)
            print(f"✅ Table {silver_table_name} created successfully!")
            print("Performing first-time load...")

            df = spark.read.table(bronze_table_name)

            columns = df.columns

            population_cols = [col_name for col_name in df.columns if "population" in col_name]

            df_unpivoted = df.melt(["borough", "cd_name"], population_cols, "year", "population")

            final_df = df_unpivoted.withColumn("borough", upper(col("borough")))\
                                  .withColumn("cd_name", upper(col("cd_name")))\
                                  .withColumn("year", regexp_replace(col("year"), "^_|_population", ""))\
                                  .withColumn("year", col("year").cast("int"))\
                                  .withColumn("population", col("population").cast("int"))\
                                  .withColumn("source", lit(bronze_table_name))\
                                  .withColumn("ingestion_timestamp", lit(ingestion_timestamp))

            final_df.write.mode("overwrite").saveAsTable(silver_table_name)
            print(f"✅ First-time load completed! {final_df.count()} rows loaded successfully from {bronze_table_name} to {silver_table_name}")


    def dim_restaurant(self):
        """Load NYC Restaurant data into the Silver layer with incremental loading."""
        
        print("🔹 Starting silver layer data ingestion for NYC Restaurant data...")

        bronze_table_name = f"{self.catalog_name}.{self.bronze_db_name}.nyc_restaurant_inspection_raw"
        silver_table_name = f"{self.catalog_name}.{self.db_name}.dim_restaurant"
        dim_restaurant_storage_path = self.silver_path + "dim_restaurant/"
        ingestion_timestamp = datetime.utcnow()

        bronze_data = spark.sql(f"""
                                    SELECT DISTINCT
                                        CAST(CAMIS AS INT) AS restaurant_id,
                                        UPPER(DBA) AS restaurant_name,
                                        NVL(UPPER(CUISINE_DESCRIPTION), 'NOT AVAILABLE') AS cuisine_type,
                                        NVL(PHONE, 'NOT AVAILABLE') AS phone_no,
                                        BUILDING AS building_no, 
                                        STREET AS street_name,
                                        CASE WHEN UPPER(BORO) = '0' THEN 'NOT AVAILABLE' ELSE UPPER(BORO) END AS borough,
                                        ZIPCODE AS zipcode,
                                        CAST(longitude AS DOUBLE) AS longitude,
                                        CAST(latitude AS DOUBLE) AS latitude
                                    FROM {bronze_table_name}
                                """)

        if self.table_exists(silver_table_name):
            print(f"📌 Table {silver_table_name} exists. Performing incremental load...")

            df = bronze_data.withColumn("source", lit(bronze_table_name))\
                        .withColumn("ingestion_timestamp", lit(ingestion_timestamp))

            df.createOrReplaceTempView("temp_view")

            existing_df = spark.table(silver_table_name)
            existing_df = existing_df.drop("source", "ingestion_timestamp", "start_date", "end_date")
            print(f"🔍 Existing record count: {existing_df.count()}")
            new_df = bronze_data.exceptAll(existing_df)
            new_records_count = new_df.count()
            print(f"🆕 New records to insert: {new_records_count}")

            if new_records_count > 0:
                spark.sql(f"""
                                MERGE INTO {silver_table_name} AS TARGET
                                USING temp_view AS SOURCE
                                ON TARGET.restaurant_id = SOURCE.restaurant_id AND TARGET.END_DATE IS NULL
                                WHEN MATCHED AND TARGET.INGESTION_TIMESTAMP < SOURCE.INGESTION_TIMESTAMP THEN 
                                UPDATE SET TARGET.END_DATE = CURRENT_TIMESTAMP()
                                WHEN NOT MATCHED THEN 
                                INSERT (
                                    restaurant_id, restaurant_name, cuisine_type, phone_no, building_no, 
                                    street_name, borough, zipcode, longitude, latitude, source, ingestion_timestamp, start_date, end_date
                                ) 
                                VALUES (
                                    SOURCE.restaurant_id, SOURCE.restaurant_name, SOURCE.cuisine_type, SOURCE.phone_no, 
                                    SOURCE.building_no, SOURCE.street_name, SOURCE.borough, SOURCE.zipcode, 
                                    SOURCE.longitude, SOURCE.latitude, '{bronze_table_name}', 
                                    {ingestion_timestamp}, current_timestamp(), null
                                )
                            """)
                print(f"✅ {new_records_count} new records inserted into {silver_table_name}.")
            else:
                print("✅ No new records to insert. Data is already up-to-date.")

        else:
            print(f"🛠️ Table {silver_table_name} does not exist. Creating Table...")
            spark.sql(f"""
                           CREATE TABLE IF NOT EXISTS {silver_table_name} (
                                restaurant_id INT,
                                restaurant_name STRING,
                                cuisine_type STRING,
                                phone_no STRING,
                                building_no STRING,
                                street_name STRING,
                                borough STRING,
                                zipcode STRING,
                                longitude DOUBLE,
                                latitude DOUBLE,
                                source STRING,
                                ingestion_timestamp TIMESTAMP,
                                start_date TIMESTAMP,
                                end_date TIMESTAMP)
                            USING DELTA
                            LOCATION '{dim_restaurant_storage_path}'
                        """)
            print(f"✅ Table {silver_table_name} created successfully!")
            print("Performing first-time load...")

            df = bronze_data.withColumn("source", lit(bronze_table_name))\
                            .withColumn("ingestion_timestamp", lit(ingestion_timestamp))\
                            .withColumn("start_date", current_timestamp())\
                            .withColumn("end_date", lit(None).cast("timestamp"))

            df.write.mode("overwrite").saveAsTable(silver_table_name)
            print(f"✅ First-time load completed! {df.count()} rows loaded successfully from {bronze_table_name} to {silver_table_name}")
    
    def dim_voilation(self):

        """Load NYC Voilation data into the Silver layer with incremental loading."""
        
        print("🔹 Starting silver layer data ingestion for NYC Voilation codes data...")

        bronze_table_name = f"{self.catalog_name}.{self.bronze_db_name}.nyc_restaurant_inspection_raw"
        silver_table_name = f"{self.catalog_name}.{self.db_name}.dim_voilation"
        dim_voilation_storage_path = self.silver_path + "dim_voilation/"
        ingestion_timestamp = datetime.utcnow()

        bronze_data = spark.sql(f"""
                                    SELECT VIOLATION_CODE, 
                                    MAX(UPPER(VIOLATION_DESCRIPTION)) AS VIOLATION_DESCRIPTION
                                    FROM {bronze_table_name}
                                    WHERE VIOLATION_CODE IS NOT NULL
                                    GROUP BY VIOLATION_CODE
                                """)

        if self.table_exists(silver_table_name):
            print(f"📌 Table {silver_table_name} exists. Performing incremental load...")

            df = bronze_data.withColumn("source", lit(bronze_table_name))\
                        .withColumn("ingestion_timestamp", lit(ingestion_timestamp))

            df.createOrReplaceTempView("temp_view")

            existing_df = spark.table(silver_table_name)
            existing_df = existing_df.drop("source", "ingestion_timestamp", "start_date", "end_date")
            print(f"🔍 Existing record count: {existing_df.count()}")
            new_df = bronze_data.exceptAll(existing_df)
            new_records_count = new_df.count()
            print(f"🆕 New records to insert: {new_records_count}")

            if new_records_count > 0:
                spark.sql(f"""
                                MERGE INTO {silver_table_name} AS TARGET
                                USING temp_view AS SOURCE
                                ON TARGET.voilation_code = SOURCE.voilation_code AND TARGET.END_DATE IS NULL
                                WHEN MATCHED AND TARGET.INGESTION_TIMESTAMP < SOURCE.INGESTION_TIMESTAMP THEN 
                                UPDATE SET TARGET.END_DATE = CURRENT_TIMESTAMP()
                                WHEN NOT MATCHED THEN 
                                INSERT (
                                    voilation_code, voilation_description, source, ingestion_timestamp, start_date, end_date
                                ) 
                                VALUES (
                                    SOURCE.voilation_code, SOURCE.voilation_description, '{bronze_table_name}', 
                                    {ingestion_timestamp}, current_timestamp(), null
                                )
                            """)
                print(f"✅ {new_records_count} new records inserted into {silver_table_name}.")
            else:
                print("✅ No new records to insert. Data is already up-to-date.")

        else:
            print(f"🛠️ Table {silver_table_name} does not exist. Creating Table...")
            spark.sql(f"""
                           CREATE TABLE IF NOT EXISTS {silver_table_name} (
                                violation_code STRING,
                                violation_description STRING,
                                source STRING,
                                ingestion_timestamp TIMESTAMP,
                                start_date TIMESTAMP,
                                end_date TIMESTAMP)
                            USING DELTA
                            LOCATION '{dim_voilation_storage_path}'
                        """)
            print(f"✅ Table {silver_table_name} created successfully!")
            print("Performing first-time load...")

            df = bronze_data.withColumn("source", lit(bronze_table_name))\
                            .withColumn("ingestion_timestamp", lit(ingestion_timestamp))\
                            .withColumn("start_date", current_timestamp())\
                            .withColumn("end_date", lit(None).cast("timestamp"))

            df.write.mode("overwrite").saveAsTable(silver_table_name)
            print(f"✅ First-time load completed! {df.count()} rows loaded successfully from {bronze_table_name} to {silver_table_name}")
    
    def fact_restaurant_inspections(self):

        """Load NYC Restaurant Inspection data into the Silver layer with incremental loading."""
        
        print("🔹 Starting silver layer data ingestion for NYC Restaurant Inspection data...")

        bronze_table_name = f"{self.catalog_name}.{self.bronze_db_name}.nyc_restaurant_inspection_raw"
        silver_table_name = f"{self.catalog_name}.{self.db_name}.fact_restaurant_inspections"
        fact_restaurant_inspections_storage_path = self.silver_path + "fact_restaurant_inspections/"
        ingestion_timestamp = datetime.utcnow()

        bronze_data = spark.sql(f"""
                                    SELECT 
                                    camis as restaurant_id,
                                    coalesce(violation_code, 'NO VIOLATIONS') as violation_code, 
                                    cast(inspection_date as DATE) as inspection_date,
                                    coalesce(upper(inspection_type), 'NONE') as inspection_type,
                                    upper(critical_flag) as critical_flag,
                                    coalesce(upper(action), 'NONE') as action, 
                                    cast(score as INT) as score, 
                                    coalesce(grade, 'NOT AVAILABLE') as grade,
                                    cast(grade_date as DATE) as grade_date,
                                    cast(record_date as DATE) as record_date
                                    FROM {bronze_table_name}
                                """)

        if self.table_exists(silver_table_name):
            print(f"📌 Table {silver_table_name} exists. Performing incremental load...")

            key_columns = [col for col in bronze_data.columns if col != "ingestion_timestamp"]

            df = (bronze_data
                    .withColumn("unique_hash", sha2(concat_ws("|", *[bronze_data[col] for col in key_columns]), 256))
                    .withColumn("source", lit(bronze_table_name))
                    .withColumn("ingestion_timestamp", lit(ingestion_timestamp))
                    .dropDuplicates(["unique_hash"])
                )

            df.createOrReplaceTempView("temp_view")

            existing_df = spark.table(silver_table_name)
            existing_df = existing_df.drop("source", "ingestion_timestamp")
            bronze_df = df.drop("source", "ingestion_timestamp")
            print(f"🔍 Existing record count: {existing_df.count()}")
            new_df = bronze_df.exceptAll(existing_df)
            new_records_count = new_df.count()
            print(f"🆕 New records to insert: {new_records_count}")

            if new_records_count > 0:
                spark.sql(f"""
                                MERGE INTO {silver_table_name} AS TARGET
                                USING temp_view AS SOURCE
                                ON TARGET.unique_hash = SOURCE.unique_hash
                                WHEN MATCHED AND TARGET.INGESTION_TIMESTAMP < SOURCE.INGESTION_TIMESTAMP THEN 
                                    UPDATE SET *
                                WHEN NOT MATCHED THEN 
                                INSERT *
                            """)
                print(f"✅ {new_records_count} new records inserted into {silver_table_name}.")
            else:
                print("✅ No new records to insert. Data is already up-to-date.")

        else:
            print(f"🛠️ Table {silver_table_name} does not exist. Creating Table...")
            spark.sql(f"""
                           CREATE TABLE IF NOT EXISTS {silver_table_name} (
                                restaurant_id STRING,
                                violation_code STRING,
                                inspection_date DATE,
                                inspection_type STRING,
                                critical_flag STRING,
                                action STRING,
                                score INT,
                                grade STRING,
                                grade_date DATE,
                                record_date DATE,
                                unique_hash STRING,
                                source STRING,
                                ingestion_timestamp TIMESTAMP)
                            USING DELTA
                            LOCATION '{fact_restaurant_inspections_storage_path}'
                        """)
            print(f"✅ Table {silver_table_name} created successfully!")
            print("Performing first-time load...")

            key_columns = [col for col in bronze_data.columns if col != "ingestion_timestamp"]

            # Generate Unique Hash Column using SHA-256
            df = (bronze_data
                    .withColumn("unique_hash", sha2(concat_ws("|", *[bronze_data[col] for col in key_columns]), 256))
                    .withColumn("source", lit(bronze_table_name))
                    .withColumn("ingestion_timestamp", lit(ingestion_timestamp))
                    .dropDuplicates(["unique_hash"])
                )

            df.write.mode("overwrite").saveAsTable(silver_table_name)
            print(f"✅ First-time load completed! {df.count()} rows loaded successfully from {bronze_table_name} to {silver_table_name}")

    def create_silver_tables(self):

        setup = Setup("silver_db")
        setup.create_db()

        self.dim_restaurant()
        self.dim_borough_population()
        self.dim_voilation()
        self.fact_restaurant_inspections()

        print(f"🎯 Data processing for Silver completed successfully.\n")
